In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, PassThrough, InverseKinematics, Solve, FixedOffsetFrame, Sphere, Rgba

import time

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [4]:
# import logging
# drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.0
x_fac = 0.5
locs = [ [0,0,0], 
        [x_fac, 1.4*sp,0.4], [x_fac,-1.4*sp,0.4], [-x_fac,-1.4*sp,0.4], [-x_fac,1.4,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1

end_effector_frame = plant.GetBodyByName("iiwa_link_7").body_frame()
desired_frame = plant.AddFrame(
    FixedOffsetFrame("desired_pos_frame", 
                     end_effector_frame, 
                     RigidTransform(p=np.array([0,0,0.1]))
                    )
)

plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(plant.num_positions())

do_viz = True

meshcat = StartMeshcat()
meshcat.Delete()
visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
diagram.ForcedPublish(diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [47]:
ik = InverseKinematics(plant, plant_context, with_joint_limits = True)
q0 = plant.GetPositions(plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(
            1e-5, 1e-5)

cur_pos = desired_frame.CalcPoseInWorld(plant_context).translation()

eps = 1e-8
pos_constraint = ik.AddPositionConstraint(end_effector_frame, 
                                            np.zeros(3),
                                            plant.world_frame(),
                                            cur_pos - eps*np.ones(3),
                                            cur_pos + eps*np.ones(3))

In [48]:
from pydrake.all import Sphere, Rgba
sphere = Sphere(0.03)
meshcat.SetObject("desired_xyz", sphere, Rgba(0,1,0,1))
meshcat.SetTransform("desired_xyz", RigidTransform(p=cur_pos))

## Set up the sliders so we can move the plant around manually

In [70]:
pos_sliders = []
pos_low = [-1, -1, 0]
pos_high = [1, 1, 1.5]
names = ["x", "y", "z"]
for i in range(3):
    pos_sliders.append(
        widgets.FloatSlider(
            min = pos_low[i], 
            max = pos_high[i], 
            value = cur_pos[i], 
            description = names[i]
        )
    )

joint_sliders = []
for i in range(plant.num_positions()):
    joint_sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=q0[i], description=f'q{i}'))

xyz = desired_frame.CalcPoseInWorld(plant_context).translation().copy()
q = plant.GetPositions(plant_context).copy()
def handle_pos_slider_change(change, idx):
    xyz[idx] = change["new"]


    ik.get_mutable_prog().SetInitialGuess(ik.q(),plant.GetPositions(plant_context))
    pos_constraint.evaluator().UpdateLowerBound(xyz - eps*np.ones(3))
    pos_constraint.evaluator().UpdateUpperBound(xyz + eps*np.ones(3))

    result = Solve(ik.prog())
    q = result.GetSolution(ik.q())

    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)
    meshcat.SetTransform("desired_xyz", RigidTransform(p=xyz))
    
def handle_joint_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    meshcat.SetTransform("desired_xyz", RigidTransform(p=desired_frame.CalcPoseInWorld(plant_context).translation()))
    diagram.ForcedPublish(diagram_context)
    
    
for idx,slider in enumerate(pos_sliders):
    slider.observe(partial(handle_pos_slider_change, idx = idx), names='value')
    
for idx,slider in enumerate(joint_sliders):
    slider.observe(partial(handle_joint_slider_change, idx = idx), names='value')

for slider in pos_sliders+joint_sliders:
    display(slider)


# visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [76]:
print(repr(np.round(plant.GetPositions(plant_context),2)))

array([ 2.22, -2.09, -1.61, -1.61, -2.87,  0.64,  3.05])
